In [1]:
from google.cloud import vision
from pathlib import Path
import cv2
import io
from PIL import Image

In [2]:
"""
Memory-efficient PDF-to-Image conversion.
"""

import subprocess as sp
import tempfile
from glob import glob

import cv2 as cv
import numpy as np


class PdfImages:
    """
    A container that holds images of PDF pages. It is a generic python sequence that
    supports `len()` and indexing.
    """

    def __init__(self, path: str, dpi: int = 200, password: str = ""):
        """
        Converts all pages to images and stores them in a temporary directory.

        Args:
            path: str
                Path to PDF file
            dpi: int
                Dots per inch (quality of image)
            password: str
                Password to decrypt encrypted PDFs
        """
        self.tempdir = tempfile.TemporaryDirectory()
        tempdir_name = self.tempdir.name
        sp.run(
            [
                "pdftoppm",
                str(path),
                f"{tempdir_name}/out",
                "-jpeg",
                "-r",
                str(dpi),
                "-upw",
                str(password),
                "-opw",
                str(password),
            ],
            check=True,
        )
        self.images = sorted(glob(f"{tempdir_name}/*.jpg"))

    def __del__(self):
        """Removes the temporary directory where the page-wise images are stored."""
        self.tempdir.cleanup()

    def __len__(self) -> int:
        """Returns the number of pages."""
        return len(self.images)

    def __getitem__(self, index: int) -> np.ndarray:
        """Returns the image of the particular page/index."""
        return cv.imread(self.images[index])

In [3]:
def construct_vision_image(image):
    """
    Construct Google Vision Image object from various types of input.
    """
    # check if path
    # try:
    ext = Path(image).suffix.lower()
    if ext == ".pdf":
        image = PdfImages(image)[0]
        content = cv2.imencode(".jpg", image)[1].tostring()
        return vision.Image(content=content)
    elif ext in [".jpg", ".jpeg", ".png", ".gif", ".bmp", ".webp", ".tiff"]:
        content = io.open(image, "rb").read()
        return vision.Image(content=content)
    # except (AttributeError, OSError):
    #     print("Invalid image path")
    #     print_traceback()

    # check if PIL
    if isinstance(image, Image.Image):
        with io.BytesIO() as buffer:
            image.save(buffer, format="JPEG")
            return vision.types.Image(content=buffer.getvalue())

In [4]:
# k= _get_image(pdf_path)

In [5]:
# # Usage example
pdf_path = "/Users/avinash/Desktop/Personal projects/ocr_to_layout-text/test_dataset/Demo Documents/Freight Invoice/1270802_EMLM_CROWN_IMPORTS_LLC-1.pdf"
# image = construct_vision_image(pdf_path)

In [6]:
from final_structure.utils import _get_image

In [7]:
k = _get_image(pdf_path)
image = vision.Image(content=k[0][0])

In [8]:
client = vision.ImageAnnotatorClient()
response =client.document_text_detection(image=image)

In [9]:
# items = []
# lines = {}
# y_threshold = 30  # Set the threshold value for y-axis alignment

# for text in response.text_annotations[1:]:
#     top_x_axis = text.bounding_poly.vertices[0].x
#     top_y_axis = text.bounding_poly.vertices[0].y
#     bottom_y_axis = text.bounding_poly.vertices[3].y

#     # Find an existing line that this text could belong to
#     found_line = None
#     for s_top_y_axis, s_item in lines.items():
#         if abs(s_top_y_axis - top_y_axis) <= y_threshold:
#             if top_y_axis < s_item[0][1] + y_threshold:
#                 found_line = s_top_y_axis
#                 break

#     if found_line is None:
#         # No suitable line found, create a new line
#         lines[top_y_axis] = [(top_y_axis, bottom_y_axis), []]
#         found_line = top_y_axis
#     else:
#         # Update the bottom_y_axis if necessary
#         _, current_bottom_y = lines[found_line][0]
#         if bottom_y_axis > current_bottom_y:
#             lines[found_line][0] = (top_y_axis, bottom_y_axis)

#     # Add the text to the found line
#     lines[found_line][1].append((top_x_axis, text.description))

# # Sort and join the texts for each line with adjusted spacing
# for _, item in lines.items():
#     if item[1]:
#         words = sorted(item[1], key=lambda t: t[0])
#         sentence = []
#         last_x = None
#         for x, word in words:
#             if last_x is not None:
#                 # Calculate space width based on the difference in x positions
#                 space_width = int((x - last_x) / 30)  # Adjust divisor to scale space width
#                 sentence.append(' ' * max(space_width, 1) + word)
#             else:
#                 sentence.append(word)
#             last_x = x + len(word) * 7  # Estimate the end x position of the current word

#         items.append((item[0], ''.join(sentence), words))

In [10]:
# items = []
# lines = {}
# y_threshold = 20  # Set the threshold value for y-axis alignment
# x_segment_width = 10  # Define the width of each horizontal segment
# max_segment_length = 50  # Maximum characters in a segment before text wraps to a new line

# for text in response.text_annotations[1:]:
#     top_x_axis = text.bounding_poly.vertices[0].x
#     top_y_axis = text.bounding_poly.vertices[0].y
#     bottom_y_axis = text.bounding_poly.vertices[3].y

#     # Find an existing line that this text could belong to
#     found_line = None
#     for s_top_y_axis, s_item in lines.items():
#         if abs(s_top_y_axis - top_y_axis) <= y_threshold:
#             if top_y_axis < s_item[0][1] + y_threshold:
#                 found_line = s_top_y_axis
#                 break

#     if found_line is None:
#         # No suitable line found, create a new line
#         lines[top_y_axis] = [(top_y_axis, bottom_y_axis), {}]
#         found_line = top_y_axis
#     else:
#         # Update the bottom_y_axis if necessary
#         _, current_bottom_y = lines[found_line][0]
#         if bottom_y_axis > current_bottom_y:
#             lines[found_line][0] = (top_y_axis, bottom_y_axis)

#     # Determine the segment for the text based on the x-coordinate
#     segment_index = top_x_axis // x_segment_width
#     if segment_index not in lines[found_line][1]:
#         lines[found_line][1][segment_index] = []

#     # Add the text to the appropriate segment
#     lines[found_line][1][segment_index].append((top_x_axis, text.description))

# # Sort and join the texts for each line and segment, handling overflow
# for _, item in lines.items():
#     sorted_segments = sorted(item[1].items())
#     full_line = []
#     last_segment_end = 0
#     for segment_index, words in sorted_segments:
#         segment_start = segment_index * x_segment_width
#         if segment_start > last_segment_end:
#             full_line.append(' ' * ((segment_start - last_segment_end) // 7))
#         sorted_words = sorted(words, key=lambda t: t[0])
#         segment_text = ' '.join(word for _, word in sorted_words)
        
#         # Check for overflow and handle by adding new lines
#         while len(segment_text) > max_segment_length:
#             cut_point = segment_text.rfind(' ', 0, max_segment_length)
#             if cut_point == -1:  # No space found, force cut
#                 cut_point = max_segment_length
#             full_line.append(segment_text[:cut_point])
#             full_line.append('\n' + ' ' * (segment_start // 7))  # New line with indentation
#             segment_text = segment_text[cut_point:].strip()
        
#         full_line.append(segment_text)
#         last_segment_end = segment_start + len(segment_text) * 7

#     items.append((item[0], ''.join(full_line)))

# # Output the formatted text
# for item in items:
#     print(item[1])

In [11]:
# items = []
# lines = {}
# y_threshold = 20  # Set the threshold value for y-axis alignment
# num_segments = 20  # Number of horizontal segments
# max_x_coordinate = max(text.bounding_poly.vertices[1].x for text in response.text_annotations[1:])  # Find the maximum x-coordinate
# segment_width = max_x_coordinate // num_segments  # Calculate segment width dynamically
# last_bottom_y = 0  # Track the bottom y-coordinate of the last processed line
# x_close_threshold = 120  # Threshold for x-axis closeness to consider merging words into the same block

# # Additional structure to track the last word's x-coordinate in each line
# last_word_x = {}

# for text in response.text_annotations[1:]:
#     top_x_axis = text.bounding_poly.vertices[0].x
#     top_y_axis = text.bounding_poly.vertices[0].y
#     bottom_y_axis = text.bounding_poly.vertices[3].y

#     # Find an existing line that this text could belong to
#     found_line = None
#     for s_top_y_axis, s_item in lines.items():
#         if abs(s_top_y_axis - top_y_axis) <= y_threshold:
#             if top_y_axis < s_item[0][1] + y_threshold:
#                 found_line = s_top_y_axis
#                 break

#     if found_line is None:
#         # No suitable line found, create a new line
#         lines[top_y_axis] = [(top_y_axis, bottom_y_axis), {}]
#         found_line = top_y_axis
#         last_word_x[found_line] = 0  # Initialize the last word x-coordinate for this line
#     else:
#         # Update the bottom_y_axis if necessary
#         _, current_bottom_y = lines[found_line][0]
#         if bottom_y_axis > current_bottom_y:
#             lines[found_line][0] = (top_y_axis, bottom_y_axis)

#     # Determine the segment for the text based on the x-coordinate
#     segment_index = top_x_axis // segment_width

#     # Check if the word is too close to the last word in the same line
#     if (top_x_axis - last_word_x[found_line]) < x_close_threshold:
#         # Find the segment of the last word in the same line
#         for seg_index, words in lines[found_line][1].items():
#             if words and words[-1][0] == last_word_x[found_line]:
#                 segment_index = seg_index  # Adjust the segment index to the last word's segment
#                 break

#     if segment_index not in lines[found_line][1]:
#         lines[found_line][1][segment_index] = []

#     # Add the text to the appropriate segment
#     lines[found_line][1][segment_index].append((top_x_axis, text.description))
#     last_word_x[found_line] = top_x_axis  # Update the last word x-coordinate for this line

# # Sort and join the texts for each line and segment
# sorted_lines = sorted(lines.items())
# for top_y_axis, item in sorted_lines:
#     sorted_segments = sorted(item[1].items())
#     full_line = []
#     last_segment_end = 0
#     for segment_index, words in sorted_segments:
#         segment_start = segment_index * segment_width
#         if segment_start > last_segment_end and (segment_start - last_segment_end) // 7 > 1:
#             full_line.append(' ' * ((segment_start - last_segment_end) // 7))
#         sorted_words = sorted(words, key=lambda t: t[0])
#         line_text = ' '.join(word for _, word in sorted_words)
#         full_line.append(line_text)
#         last_segment_end = segment_start + len(line_text) * 7

#     items.append((item[0], ''.join(full_line)))

#     # Check for large vertical gaps and add an extra line if necessary
#     if last_bottom_y and (top_y_axis - last_bottom_y > y_threshold * 2):
#         items.append((last_bottom_y + y_threshold, ''))
#     last_bottom_y = item[0][1]

# # Output the formatted text
# for item in items:
#     print(item[1])

In [14]:
def format_text_from_annotations(response,exclusion_zones):
    items = []
    lines = {}
    table_annotated = {}
    y_threshold = 10  # Set the threshold value for y-axis alignment
    num_segments = 30  # Number of horizontal segments
    max_x_coordinate = max(text.bounding_poly.vertices[1].x for text in response.text_annotations[1:])  # Find the maximum x-coordinate
    segment_width = max_x_coordinate // num_segments  # Calculate segment width dynamically
    last_bottom_y = 0  # Track the bottom y-coordinate of the last processed line
    x_close_threshold = 50  # Threshold for x-axis closeness to consider merging words into the same block

    # Additional structure to track the last word's x-coordinate in each line
    last_word_x = {}
    def is_within_exclusion_zone(x1, y1, x2, y2):
        for idx,ex_zone in exclusion_zones.items():
            (ex_x1, ex_y1, ex_x2, ex_y2) = ex_zone
            if not (x2 < ex_x1 or x1 > ex_x2 or y2 < ex_y1 or y1 > ex_y2):
                return (True,idx)
        return (False,False)

    for text in response.text_annotations[1:]:
        top_x_axis = text.bounding_poly.vertices[0].x
        top_y_axis = text.bounding_poly.vertices[0].y
        bottom_y_axis = text.bounding_poly.vertices[3].y
        is_present = is_within_exclusion_zone(top_x_axis, top_y_axis, text.bounding_poly.vertices[1].x, bottom_y_axis)
        if is_present[0]:
            if is_present[1] not in table_annotated:
                table_annotated[is_present[1]] = 1
                text.description = len(text.description) * " "
            else:
                if table_annotated[is_present[1]] == 3:
                    text.description = f"# Table_{is_present[1]} is here"
                    table_annotated[is_present[1]] += 1
                else:
                    text.description = len(text.description) * " "
                    table_annotated[is_present[1]] += 1

        # Find an existing line that this text could belong to
        found_line = None
        for s_top_y_axis, s_item in lines.items():
            if abs(s_top_y_axis - top_y_axis) <= y_threshold:
                if top_y_axis < s_item[0][1] + y_threshold:
                    found_line = s_top_y_axis
                    break
        print(table_annotated)
        if found_line is None:
            # No suitable line found, create a new line
            lines[top_y_axis] = [(top_y_axis, bottom_y_axis), {}]
            found_line = top_y_axis
            last_word_x[found_line] = 0  # Initialize the last word x-coordinate for this line
        else:
            # Update the bottom_y_axis if necessary
            _, current_bottom_y = lines[found_line][0]
            if bottom_y_axis > current_bottom_y:
                lines[found_line][0] = (top_y_axis, bottom_y_axis)
        # Determine the segment for the text based on the x-coordinate
        segment_index = top_x_axis // segment_width

        # Check if the word is too close to the last word in the same line
        if (top_x_axis - last_word_x[found_line]) < x_close_threshold:
            # Find the segment of the last word in the same line
            for seg_index, words in lines[found_line][1].items():
                if words and words[-1][0] == last_word_x[found_line]:
                    segment_index = seg_index  # Adjust the segment index to the last word's segment
                    break

        if segment_index not in lines[found_line][1]:
            lines[found_line][1][segment_index] = []

        # Add the text to the appropriate segment
        lines[found_line][1][segment_index].append((top_x_axis, text.description))
        last_word_x[found_line] = top_x_axis  # Update the last word x-coordinate for this line

    # Sort and join the texts for each line and segment
    sorted_lines = sorted(lines.items())
    for top_y_axis, item in sorted_lines:
        sorted_segments = sorted(item[1].items())
        full_line = []
        last_segment_end = 0
        for segment_index, words in sorted_segments:
            segment_start = segment_index * segment_width
            if segment_start > last_segment_end and (segment_start - last_segment_end) // 7 > 1:
                full_line.append(' ' * ((segment_start - last_segment_end) // 7))
            sorted_words = sorted(words, key=lambda t: t[0])
            line_text = ' '.join(word for _, word in sorted_words)
            full_line.append(line_text)
            last_segment_end = segment_start + len(line_text) * 7

        items.append((item[0], ''.join(full_line)))

        # Check for large vertical gaps and add an extra line if necessary
        if last_bottom_y and (top_y_axis - last_bottom_y > y_threshold * 2):
            items.append((last_bottom_y + y_threshold, ''))
        last_bottom_y = item[0][1]

    # Combine all lines into a single string
    formatted_text = '\n'.join(item[1] for item in items)
    return formatted_text

In [15]:
print(format_text_from_annotations(response,{"1":(259, 58, 575, 98),"2":(259, 98, 575, 229),"3":(24, 280, 575, 586)}))

{}
{}
{}
{}
{'1': 1}
{'1': 2}
{'1': 3}
{'1': 4}
{'1': 5}
{'1': 6}
{'1': 7}
{'1': 7}
{'1': 7}
{'1': 7}
{'1': 8}
{'1': 9}
{'1': 10}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 11}
{'1': 12}
{'1': 12, '2': 1}
{'1': 12, '2': 2}
{'1': 12, '2': 3}
{'1': 13, '2': 3}
{'1': 13, '2': 4}
{'1': 13, '2': 5}
{'1': 13, '2': 6}
{'1': 13, '2': 7}
{'1': 13, '2': 8}
{'1': 13, '2': 9}
{'1': 13, '2': 10}
{'1': 13, '2': 11}
{'1': 13, '2': 12}
{'1': 13, '2': 13}
{'1': 13, '2': 14}
{'1': 13, '2': 15}
{'1': 13, '2': 16}
{'1': 13, '2': 17}
{'1': 13, '2': 18}
{'1': 13, '2': 19}
{'1': 13, '2': 20}
{'1': 13, '2': 21}
{'1': 13, '2': 22}
{'1': 13, '2': 23}
{'1': 13, '2': 24}
{'1': 13, '2': 25}
{'1': 13, '2': 26}
{'1': 13, '2': 27}
{'1': 13, '2': 28}
{'1': 13, '2': 28}
{'1': 13, '2': 28}
{'1': 13, '2': 28}
{'1': 13, '2': 28}
{'1': 13, '2': 28}
{'1': 13, '2': 28}
{'1':

In [ ]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

def string_to_pdf(text, width=1800, height=1500):  # Default size for A4
    # Create a new PDF with custom dimensions
    pdfmetrics.registerFont(TTFont('Monaco', 'MONACO.TTF')) 
    c = canvas.Canvas("output.pdf", pagesize=(width, height))
    
    c.setFont("Monaco", 12)
    # Set the font and size
    # c.setFont("Courier New'", 12)
    
    # Split the text into lines
    lines = text.split('\n')
    
    # Draw each line on the PDF
    y = height - 50  # Start drawing 50 points from the top of the page
    for line in lines:
        c.drawString(50, y, line)  # Start drawing 50 points from the left of the page
        y -= 20  # Move down 20 points for the next line
    
    # Save the PDF
    c.showPage()
    c.save()

text = ""
for item in items:
    text += item[1] + '\n'
string_to_pdf(text)

In [ ]:
page = response.full_text_annotation.pages[0]

In [ ]:
page.width

In [ ]:
page.height

In [ ]:
# from collections import defaultdict

# items = []
# lines = {}
# y_threshold = 25  # Set the threshold value for y-axis alignment
# x_positions = defaultdict(list)

# for text in response.text_annotations[1:]:
#     top_x_axis = text.bounding_poly.vertices[0].x
#     top_y_axis = text.bounding_poly.vertices[0].y
#     bottom_y_axis = text.bounding_poly.vertices[3].y

#     # Find an existing line that this text could belong to
#     found_line = None
#     for s_top_y_axis, s_item in lines.items():
#         if abs(s_top_y_axis - top_y_axis) <= y_threshold:
#             if top_y_axis < s_item[0][1] + y_threshold:
#                 found_line = s_top_y_axis
#                 break

#     if found_line is None:
#         # No suitable line found, create a new line
#         lines[top_y_axis] = [(top_y_axis, bottom_y_axis), []]
#         found_line = top_y_axis
#     else:
#         # Update the bottom_y_axis if necessary
#         _, current_bottom_y = lines[found_line][0]
#         if bottom_y_axis > current_bottom_y:
#             lines[found_line][0] = (top_y_axis, bottom_y_axis)

#     # Add the text to the found line and track x positions
#     lines[found_line][1].append((top_x_axis, text.description))
#     x_positions[top_x_axis].append(text.description)

# # Calculate maximum width for each x position
# max_widths = {x: max(len(word) for word in words) for x, words in x_positions.items()}

# # Sort and join the texts for each line with adjusted spacing and alignment
# for _, item in lines.items():
#     if item[1]:
#         words = sorted(item[1], key=lambda t: t[0])
#         sentence = []
#         last_x = None
#         for x, word in words:
#             if last_x is not None:
#                 space_width = int((x - last_x) / 30)  # Adjust divisor to scale space width
#                 sentence.append(' ' * max(space_width, 1))
#             # Pad word for alignment
#             padded_word = word.ljust(max_widths[x])
#             sentence.append(padded_word)
#             last_x = x + len(padded_word) * 7  # Estimate the end x position of the current word

#         items.append((item[0], ''.join(sentence), words))


In [ ]:
# from collections import defaultdict

# items = []
# lines = {}
# y_threshold = 20  # Adjust this value based on average text line height and spacing
# x_positions = defaultdict(list)

# # Process text annotations to organize them into lines
# for text in response.text_annotations[1:]:
#     top_x_axis = text.bounding_poly.vertices[0].x
#     top_y_axis = text.bounding_poly.vertices[0].y
#     bottom_y_axis = text.bounding_poly.vertices[3].y

#     # Determine the line by checking y-axis alignment within a threshold
#     found_line = None
#     for s_top_y_axis, s_item in lines.items():
#         line_mid_y = (s_item[0][0] + s_item[0][1]) / 2
#         if abs(line_mid_y - (top_y_axis + bottom_y_axis) / 2) <= y_threshold:
#             found_line = s_top_y_axis
#             break

#     if found_line is None:
#         lines[top_y_axis] = [(top_y_axis, bottom_y_axis), []]
#         found_line = top_y_axis
#     else:
#         # Update the line's bounding box if necessary
#         existing_top, existing_bottom = lines[found_line][0]
#         new_top = min(existing_top, top_y_axis)
#         new_bottom = max(existing_bottom, bottom_y_axis)
#         lines[found_line][0] = (new_top, new_bottom)

#     lines[found_line][1].append((top_x_axis, text.description))
#     x_positions[top_x_axis].append(text.description)

# # Calculate maximum width for each x position
# max_widths = {x: max(len(word) for word in words) for x, words in x_positions.items()}

# # Sort and join the texts for each line with adjusted spacing and alignment
# for _, item in lines.items():
#     if item[1]:
#         words = sorted(item[1], key=lambda t: t[0])
#         sentence = []
#         last_end_x = 0  # Track the end position of the last word

#         for x, word in words:
#             if last_end_x != 0:
#                 space_width = (x - last_end_x) // 15
#                 sentence.append(' ' * max(space_width, 1))
#             padded_word = word.ljust(max_widths[x])
#             sentence.append(padded_word)
#             last_end_x = x + len(padded_word) * 6

#         items.append((item[0], ''.join(sentence), words))

# # Align text starting at the same x-axis position
# max_line_length = max(len(line[1]) for line in items)
# aligned_text = []
# for _, text, _ in items:
#     aligned_text.append(text.ljust(max_line_length))

# print(aligned_text)

In [ ]:
x_positions

In [ ]:
for item in items:
    print(item[1])

In [ ]:
# from collections import defaultdict

# items = []
# lines = defaultdict(list)
# y_threshold = 10  # Adjust based on the text size and line spacing
# x_threshold = 50  # Horizontal threshold for grouping words closely

# # Process text annotations to organize them into lines
# for text in response.text_annotations[1:]:
#     top_x_axis = text.bounding_poly.vertices[0].x
#     top_y_axis = text.bounding_poly.vertices[0].y
#     bottom_y_axis = text.bounding_poly.vertices[3].y

#     # Determine the line by checking y-axis alignment within a threshold
#     found_line = None
#     for s_top_y_axis, s_item in lines.items():
#         if abs(s_top_y_axis - top_y_axis) <= y_threshold:
#             found_line = s_top_y_axis
#             break

#     if found_line is None:
#         lines[top_y_axis].append((top_x_axis, text.description))
#     else:
#         lines[found_line].append((top_x_axis, text.description))

# # Identify common x-axis starting positions
# x_starts = defaultdict(list)
# for y, texts in lines.items():
#     for x, text in texts:
#         x_starts[x].append((y, text))

# # Calculate the maximum width for each x-axis start position
# max_widths = {}
# for x, entries in x_starts.items():
#     max_widths[x] = max(len(text) for _, text in entries)

# # Sort lines and format text with horizontal alignment
# sorted_lines = sorted(lines.items(), key=lambda x: x[0])
# formatted_lines = []
# for y, texts in sorted_lines:
#     sorted_texts = sorted(texts, key=lambda x: x[0])
#     line_text = []
#     last_end_x = 0
#     for x, text in sorted_texts:
#         if last_end_x != 0:
#             space_width = (x - last_end_x) // 10
#             if space_width > x_threshold // 10:
#                 line_text.append('   ')  # Add extra space for larger gaps
#             else:
#                 line_text.append(' ')
#         # Pad text for alignment
#         padded_text = text.ljust(max_widths[x])
#         line_text.append(padded_text)
#         last_end_x = x + len(padded_text) * 7  # Estimate the end x position of the current word
#     formatted_lines.append(''.join(line_text))

# # Print each line with some separation
# for line in formatted_lines:
#     print(line)
#     print()  # Print a blank line for separation between lines